In [ ]:
import tensorflow as tf
#import matplotlib as mpl
#import matplotlib.pyplot as plt
import os
import numpy as np
import PIL

#mpl.rcParams['figure.figsize'] = (8, 8)
#mpl.rcParams['axes.grid'] = False

In [ ]:
pretrained_model = tf.keras.applications.MobileNetV2(include_top=True,
                                                     weights='imagenet')
pretrained_model.trainable = False

# ImageNet labels
decode_predictions = tf.keras.applications.mobilenet_v2.decode_predictions
model_preprocess = tf.keras.applications.mobilenet_v2.preprocess_input

14536120/14536120 [==============================] - 0s 0us/step


# Util

In [ ]:
# Helper function to preprocess the image so that it can be inputted in MobileNetV2
def preprocess(image):
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, [224, 224])
    image = tf.keras.applications.mobilenet_v2.preprocess_input(image)
    image = image[None, ...]
    return image

# Helper function to extract labels from probability vector
def get_imagenet_label(probs):
    return decode_predictions(probs, top=1)[0][0]

In [ ]:
def predict_img(img_path):
    image_raw = tf.io.read_file(img_path)
    image = tf.image.decode_jpeg(image_raw)
    height, width, channels = image.shape
    image = preprocess(image)
    # image = tf.image.resize(image, [224, 224])
    # image = image[None, ...]
    image_probs = pretrained_model.predict(image)

    return (image, image_probs, height, width)


In [ ]:
loss_object = tf.keras.losses.CategoricalCrossentropy()

def create_adversarial_pattern(input_image, input_label):
    with tf.GradientTape() as tape:
        tape.watch(input_image)
        prediction = pretrained_model(input_image)
        loss = loss_object(input_label, prediction)

    # Get the gradients of the loss w.r.t to the input image.
    gradient = tape.gradient(loss, input_image)
    # Get the sign of the gradients to create the perturbation
    signed_grad = tf.sign(gradient)
    return signed_grad

In [ ]:
def get_pertubation(img, prob):
    # Get the input label of the image.
    labrador_retriever_index = 208
    label = tf.one_hot(labrador_retriever_index, prob.shape[-1])
    label = tf.reshape(label, (1, prob.shape[-1]))

    perturbations = create_adversarial_pattern(img, label)
    # plt.imshow(perturbations[0] * 0.5 + 0.5);  # To change [-1, 1] to [0,1]
    return perturbations

In [ ]:
def tensor_to_image(tensor, height, width):
    tensor = tf.image.resize(
        tensor,
        [height, width]
    )

    tensor = tensor*255
    tensor = np.array(tensor, dtype=np.uint8)
    if np.ndim(tensor)>3:
        assert tensor.shape[0] == 1
        tensor = tensor[0]
    return PIL.Image.fromarray(tensor)

# Create Mask folder

In [ ]:
CLEAN_PATH = r'C:\Users\Admin\Documents\demo'
MASK_PATH = r'C:\Users\Admin\Documents\demo'
imgs = os.listdir(CLEAN_PATH)

NameError: ignored

In [ ]:
epsilon = .05

In [ ]:
for img in imgs:
    img_path = os.path.join(CLEAN_PATH, img)

    image, prob, he, wi = predict_img(img_path)
    pertubation = get_pertubation(image, prob)
    masked_img = image + epsilon*pertubation
    masked_img = tf.clip_by_value(masked_img, -1, 1)
    # masked_img = model_preprocess(masked_img)
    masked_img = tensor_to_image(masked_img[0]*0.5 + 0.5, he, wi)
    masked_img.save(os.path.join(MASK_PATH, img))

    # masked_img = tf.squeeze(masked_img, axis=0)
    # # masked_img = tf.reverse(masked_img, axis=[-1])
    # masked_img = tf.cast(masked_img * 255, tf.uint8)
    # masked_img = tf.image.encode_jpeg(masked_img, format='rgb', quality=100)
    # tf.io.write_file(os.path.join(MASK_PATH, img), masked_img)



1/1 [==============================] - 0s 31ms/step
